# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks= pd.read_csv("sp_500_stocks-Copy1.csv")
from Api_token1 import IEX_CLOUD_API_TOKEN 


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol='AAPL'
Api_url= f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(Api_url).json()

data

{'companyName': 'Apple Inc',
 'marketcap': 2311785840725,
 'week52high': 144.97,
 'week52low': 57.85,
 'week52change': 0.7111630785494574,
 'sharesOutstanding': 16945564743,
 'float': 0,
 'avg10Volume': 131238644,
 'avg30Volume': 114535134,
 'day200MovingAvg': 122.67,
 'day50MovingAvg': 132.73,
 'employees': 0,
 'ttmEPS': 3.81,
 'ttmDividendRate': 0.8349012915346891,
 'dividendYield': 0.005976734326062981,
 'nextDividendDate': '2021-02-04',
 'exDividendDate': '2020-11-01',
 'nextEarningsDate': '2021-01-19',
 'peRatio': 36.59567671954912,
 'beta': 1.188637506615286,
 'maxChangePercent': 51.46163138761312,
 'year5ChangePercent': 5.395515107142216,
 'year2ChangePercent': 2.414947323395336,
 'year1ChangePercent': 0.780392756606491,
 'ytdChangePercent': 0.01804775848831061,
 'month6ChangePercent': 0.2815023977696238,
 'month3ChangePercent': 0.2500631348715677,
 'month1ChangePercent': 0.01816220235456172,
 'day30ChangePercent': 0.018077801401956802,
 'day5ChangePercent': -0.05973419237527771

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
ydt_change= data["year1ChangePercent"]
ydt_change

0.780392756606491

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One_Year_Price_Return', 'Number of Shares to Buy']

In [6]:
data

{'companyName': 'Apple Inc',
 'marketcap': 2311785840725,
 'week52high': 144.97,
 'week52low': 57.85,
 'week52change': 0.7111630785494574,
 'sharesOutstanding': 16945564743,
 'float': 0,
 'avg10Volume': 131238644,
 'avg30Volume': 114535134,
 'day200MovingAvg': 122.67,
 'day50MovingAvg': 132.73,
 'employees': 0,
 'ttmEPS': 3.81,
 'ttmDividendRate': 0.8349012915346891,
 'dividendYield': 0.005976734326062981,
 'nextDividendDate': '2021-02-04',
 'exDividendDate': '2020-11-01',
 'nextEarningsDate': '2021-01-19',
 'peRatio': 36.59567671954912,
 'beta': 1.188637506615286,
 'maxChangePercent': 51.46163138761312,
 'year5ChangePercent': 5.395515107142216,
 'year2ChangePercent': 2.414947323395336,
 'year1ChangePercent': 0.780392756606491,
 'ytdChangePercent': 0.01804775848831061,
 'month6ChangePercent': 0.2815023977696238,
 'month3ChangePercent': 0.2500631348715677,
 'month1ChangePercent': 0.01816220235456172,
 'day30ChangePercent': 0.018077801401956802,
 'day5ChangePercent': -0.05973419237527771

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
for stocks in symbol_strings[:1]:
    print(stocks)

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA


In [8]:
final_data= pd.DataFrame(columns= my_columns)
final_data
for stocks in symbol_strings[:1]:
    batch_Api_url= f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={stocks}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_Api_url).json() 
    for symbol in stocks.split(","):
        final_data = final_data.append(
                                        pd.Series([symbol,data[symbol]["quote"]["latestPrice"],
                                                  data[symbol]["stats"]["year1ChangePercent"],"N/A"
                         
                                            ],
                                                  index = my_columns), 
                                        ignore_index = True)
final_data

,Ticker,Price,One_Year_Price_Return,Number of Shares to Buy
0,A,128.350,0.516216,N/A
1,AAL,17.780,-0.387821,N/A
2,AAP,156.609,0.184824,N/A
3,AAPL,141.090,0.784018,N/A
4,ABBV,106.500,0.356100,N/A
...,...,...,...,...
95,CINF,88.840,-0.143279,N/A
96,CL,80.940,0.094550,N/A
97,CLX,212.801,0.325875,N/A
98,CMA,61.610,0.051555,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [9]:
final_data.sort_values('One_Year_Price_Return', ascending = False, inplace = True)
final_data = final_data[:51]
final_data.reset_index(inplace= True)
final_data


,index,Ticker,Price,One_Year_Price_Return,Number of Shares to Buy
0,78,CARR,39.930,2.237401,N/A
1,24,ALGN,568.680,1.223476,N/A
2,23,ALB,170.620,1.145077,N/A
3,31,AMD,88.990,0.921307,N/A
4,85,CDNS,140.360,0.898485,N/A
5,6,ABMD,337.509,0.882533,N/A
6,29,AMAT,104.710,0.831069,N/A
7,3,AAPL,141.090,0.784018,N/A
8,36,AMZN,3476.390,0.690452,N/A
9,45,APTV,141.700,0.671322,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")
    try:
        float(portfolio_size)
    
    except ValueError:

        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()

Enter the value of your portfolio:20000


In [11]:
position_size = float(portfolio_size) / len(final_data.index)
print(position_size)
for i in range(0, len(final_data['Ticker'])):
      final_data.loc[i,'Number of Shares to Buy']= math.floor(position_size/final_data["Price"][i])

final_data


392.15686274509807


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One_Year_Price_Return,Number of Shares to Buy
0,78,CARR,39.930,2.237401,9
1,24,ALGN,568.680,1.223476,0
2,23,ALB,170.620,1.145077,2
3,31,AMD,88.990,0.921307,4
4,85,CDNS,140.360,0.898485,2
5,6,ABMD,337.509,0.882533,1
6,29,AMAT,104.710,0.831069,3
7,3,AAPL,141.090,0.784018,2
8,36,AMZN,3476.390,0.690452,0
9,45,APTV,141.700,0.671322,2


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [12]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_data= pd.DataFrame(columns= hqm_columns)
hqm_data


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score


In [13]:

for stocks in symbol_strings:
    batch_Api_url= f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={stocks}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_Api_url).json() 
    for symbol in stocks.split(","):
            hqm_data = hqm_data.append(
                                            pd.Series([symbol,
                                                       data[symbol]["quote"]["latestPrice"],
                                                       0,
                                                      data[symbol]["stats"]["year1ChangePercent"],
                                                       0,
                                                       data[symbol]["stats"]["month6ChangePercent"],
                                                       0,
                                                       data[symbol]["stats"]["month3ChangePercent"],
                                                        0,
                                                       data[symbol]["stats"]["month1ChangePercent"],
                                                       0,
                                                       0



                                                ],
                                                      index = hqm_columns), 
                                            ignore_index = True)
hqm_data

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.150,0,0.513243,0,0.291518,0,0.172741,0,0.0432,0,0
1,AAL,17.160,0,-0.391911,0,0.493145,0,0.500773,0,0.054225,0,0
2,AAP,155.250,0,0.184646,0,0.030593,0,0.019675,0,-0.02233,0,0
3,AAPL,137.260,0,0.776566,0,0.287299,0,0.253198,0,0.017859,0,0
4,ABBV,105.230,0,0.355425,0,0.1227,0,0.189687,0,-0.024024,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.910,0,0.02553,0,0.180531,0,0.125395,0,-0.02112,0,0
501,ZBH,163.358,0,0.080331,0,0.186456,0,0.206816,0,0.03003,0,0
502,ZBRA,414.910,0,0.729892,0,0.447499,0,0.383889,0,0.05573,0,0
503,ZION,46.900,0,0.046672,0,0.454726,0,0.39222,0,0.055825,0,0


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [20]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_data.index:
        for time_period in time_periods:

            hqm_data.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_data[f'{time_period} Price Return'], hqm_data.loc[row, f'{time_period} Price Return'])/100

 


hqm_data


TypeError: '<' not supported between instances of 'NoneType' and 'float'

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: